In [18]:
import cobra
from copy import deepcopy
import pickle

# Load model

def getModel():
    model = cobra.io.read_sbml_model('yeastGEM.xml')

    # Correct metabolite ids:
    for met in model.metabolites:
        met.id = met.id.replace('__91__', '_')
        met.id = met.id.replace('__93__', '')

    model.solver = 'gurobi'
    return model



In [3]:
model = getModel()

Academic license - for non-commercial use only


In [2]:
print(len(model.reactions))
print(len(model.genes))
print(len(model.metabolites))


3949
1133
2680


In [3]:
print(model.get_metabolite_compartments())

set(['c', 'e', 'g', 'mm', 'm', 'vm', 'ce', 'n', 'p', 'lp', 'v', 'erm', 'gm', 'er'])


In [4]:
SMatrix = cobra.util.array.create_stoichiometric_matrix(model,"dense")
print(len(SMatrix))
print(len(SMatrix[0]))

2680
3949


In [8]:
print([[x.reaction, x.id] for x in model.metabolites.get_by_id("s_0681_e").reactions]) #ethanol
print([[x.reaction, x.id] for x in model.metabolites.get_by_id("s_0565_e").reactions]) #glucose
print([[x.reaction, x.id] for x in model.metabolites.get_by_id("s_0766_e").reactions]) #glycerol
print([[x.reaction, x.id] for x in model.metabolites.get_by_id("s_0458_e").reactions]) #CO2
print([[x.reaction, x.id] for x in model.metabolites.get_by_id("s_0364_e").reactions]) #Acetate

reactionsToConstrain = [model.reactions.get_by_id('r_1761'), 
                       model.reactions.get_by_id('r_1714'),
                       model.reactions.get_by_id('r_1808'),
                       model.reactions.get_by_id('r_1672'),
                       model.reactions.get_by_id('r_1634')]


[['s_0681_e --> ', 'r_1761'], ['s_0680_c <=> s_0681_e', 'r_1762']]
[['s_0805_e + s_1466_e --> s_0554_e + s_0565_e', 'r_1024'], ['s_0003_e + s_0805_e --> s_0565_e', 'r_0370'], ['s_0565_e --> s_0563_c', 'r_1166'], ['s_0805_e + s_4131_e <=> s_0554_e + s_0565_e', 'r_4420'], ['s_0565_e <=> ', 'r_1714'], ['s_0805_e + s_4140_e --> s_0565_e + s_1106_e', 'r_4400']]
[['s_0766_e + s_0796_e --> s_0765_c + s_0794_c', 'r_1171'], ['s_0765_c --> s_0766_e', 'r_1172'], ['s_0766_e --> ', 'r_1808']]
[['s_0456_c <=> s_0458_e', 'r_1697'], ['s_0458_e + s_0805_e <=> s_0446_e + s_0796_e', 'r_1668'], ['s_0458_e --> ', 'r_1672']]
[['s_0362_c <=> s_0364_e', 'r_1106'], ['s_0364_e --> ', 'r_1634']]


In [9]:
model.optimize()
fvaResult = cobra.flux_analysis.variability.flux_variability_analysis(model,reactionsToConstrain,fraction_of_optimum = 0.0,loopless=True)
model.summary()
print(fvaResult)

IN FLUXES          OUT FLUXES      OBJECTIVES
-----------------  --------------  --------------
s_1277_e  2.23     s_0805_e  3.91  r_2111  0.0879
s_0565_e  1        s_0458_e  2.41
s_0796_e  0.789
s_0420_e  0.557
s_1324_e  0.0231
s_1468_e  0.00754
        minimum   maximum
r_1761      0.0  2.000000
r_1714     -1.0 -0.034104
r_1808      0.0  1.582398
r_1672      0.0  6.000000
r_1634      0.0  2.499909


In [10]:
constrianedModel = getModel()

mediaMetabolites = ["s_0565_e", "s_0420_e", "s_1007_e","s_1030_e", "s_1551_e", "s_1022_e", 
                    "s_1277_e", "s_0452_e", "s_1154_e", "s_1406_e", "s_1374_e", "s_1438_e"]

mediaReactions = ['r_1714','r_1654','r_1893','r_1902','r_2090','r_1899','r_1992','r_1671',
                 'r_1947','r_2038','r_2020','r_2049']

maximumFluxes = [0.6993938587,6.8773538562,0.0541395486,0.0562964949,0.0749464668,
                 0.1921170999,5250,6.87650935287135E-05,9.32504440497336E-05,
                 4.46381124455309E-05,0.0002148887,0.1437371663]


newMedia = {reaction:flux for reaction,flux in zip(mediaReactions,maximumFluxes)}

constrianedModel.medium = newMedia

reactionsToConstrain = [constrianedModel.reactions.get_by_id('r_1761'), #ethanol
                       constrianedModel.reactions.get_by_id('r_1714'), #glucose
                       constrianedModel.reactions.get_by_id('r_1808'), #glyerol
                       constrianedModel.reactions.get_by_id('r_1672'), #CO2
                       constrianedModel.reactions.get_by_id('r_1634')] #acetate
outputGlycerol = .028 
outputEthanol = .34
inputGlucose = -1*.69
outputCO2 = .42

constrianedModel.reactions.get_by_id('r_1761').lower_bound = outputEthanol*.9
constrianedModel.reactions.get_by_id('r_1761').upper_bound = outputEthanol*1.1

constrianedModel.reactions.get_by_id('r_1808').lower_bound = outputGlycerol*.9
constrianedModel.reactions.get_by_id('r_1808').upper_bound = outputGlycerol*1.1

constrianedModel.reactions.get_by_id('r_1714').lower_bound = inputGlucose*1.1
constrianedModel.reactions.get_by_id('r_1714').upper_bound = inputGlucose*.9

constrianedModel.reactions.get_by_id('r_1672').lower_bound = outputCO2*1.1
constrianedModel.reactions.get_by_id('r_1672').upper_bound = outputCO2*.9

# for metabolite in mediaMetabolites:
#     print([[x.id,x.reaction] for x in constrianedModel.metabolites.get_by_id(metabolite).reactions])

constrianedModel.optimize()
fvaResult = cobra.flux_analysis.variability.flux_variability_analysis(constrianedModel,reactionsToConstrain,
                                                            fraction_of_optimum=0.0,loopless = True)

constrianedModel.summary()
print(fvaResult)

{'r_2005': 1000.0, 'r_1714': 1.0, 'r_1992': 1000.0, 'r_2060': 1000.0, 'r_1654': 1000.0, 'r_1832': 1000.0, 'r_1861': 1000.0, 'r_2100': 1000.0}
IN FLUXES        OUT FLUXES        OBJECTIVES
---------------  ----------------  ----------------
s_0565_e  0.621  s_0805_e  0.563   r_2111  6.79e-12
s_1277_e  0.258  s_0458_e  0.378
s_0420_e  0.202  s_0681_e  0.374
                 s_0723_e  0.241
                 s_1272_e  0.167
                 s_4167_e  0.132
                 s_0796_e  0.105
                 s_1521_e  0.0542
                 s_0966_e  0.0504
                 s_0170_e  0.0504
                 s_0766_e  0.0308
        minimum   maximum
r_1761   0.3060  0.374000
r_1714  -0.7590 -0.621000
r_1808   0.0252  0.030800
r_1672   0.3780  0.378000
r_1634   0.0000  1.097583


In [22]:
fullFVASol = cobra.flux_analysis.variability.flux_variability_analysis(constrianedModel,constrianedModel.reactions[:500],
                                                            fraction_of_optimum=0.0,loopless = True)
pickle.dump(fullFVASol,open("FVAYeastGEMConstrained.pkl","wb"))


In [20]:
fullFVASol = pickle.load(open("FVAYeastGEMConstrained.pkl","rb"))

In [23]:

print type(fullFVASol)

constrainedReactions = {reaction:sum([maxi/2,mini/2]) for reaction,maxi,mini in 
            zip(fullFVASol.index.values,fullFVASol.maximum.values,fullFVASol.minimum.values)
            if maxi-mini < .01}

print constrainedReactions


<class 'pandas.core.frame.DataFrame'>
{'r_0170': 0.0, 'r_0177': 0.0, 'r_0176': 0.0, 'r_0478': 0.0, 'r_0479': 0.0, 'r_0477': 0.0, 'r_0178': 0.0, 'r_0358': 0.0, 'r_0473': 0.0, 'r_0471': 0.0, 'r_0078': 0.0, 'r_0370': 0.0, 'r_0373': 0.0, 'r_0070': 0.0, 'r_0076': 0.0, 'r_0077': 0.0, 'r_0282': 0.0, 'r_0631': 0.0, 'r_0455': 0.0, 'r_0500': 0.0, 'r_0028': 0.0, 'r_0085': 0.0, 'r_0084': 0.0, 'r_0083': 0.0, 'r_0081': 0.0, 'r_0249': 0.0, 'r_0402': 0.0, 'r_0400': 0.0, 'r_0632': 0.0, 'r_0507': 0.0, 'r_0630': 0.0, 'r_0618': 0.0, 'r_0475': 0.0, 'r_0469': 0.0, 'r_0468': 0.0, 'r_0188': 0.0, 'r_0189': 0.0, 'r_0465': 0.0, 'r_0187': 0.0, 'r_0184': 0.0, 'r_0185': 0.0, 'r_0461': 0.0, 'r_0183': 0.0, 'r_0180': 0.0, 'r_0181': 0.0, 'r_0069': 0.0, 'r_0068': 0.0, 'r_0342': 0.0, 'r_0343': 0.0, 'r_0345': 0.0, 'r_0100': 0.0, 'r_0101': 0.0, 'r_0348': 0.0, 'r_0346': 0.0, 'r_0508': 0.0, 'r_0108': 0.0, 'r_0109': 0.0, 'r_0509': 0.0, 'r_0621': 0.0, 'r_0597': 0.0, 'r_0364': 0.0, 'r_0549': 0.0, 'r_0574': 0.0, 'r_0573': 0.0, '

In [ ]:
constrainedGeometricFBASol = cobra.flux_analysis.geometric_fba(constrianedModel)
geometricFBASol = cobra.flux_analysis.geometric_fba(model)

cobra/util/solver.py:408 UserWarning: solver status is 'infeasible'
